In [4]:
#setup kaggle API key as env variables
import json
import os
kaggle_credentails = json.load(open('/content/kaggle.json'))
os.environ['KAGGLE_USERNAME'] = kaggle_credentails['username']
os.environ['KAGGLE_KEY'] = kaggle_credentails['key']

In [5]:
!kaggle datasets download -d abdallahalidev/plantvillage-dataset

Dataset URL: https://www.kaggle.com/datasets/abdallahalidev/plantvillage-dataset
License(s): CC-BY-NC-SA-4.0
 99% 2.01G/2.04G [00:14<00:00, 251MB/s]
100% 2.04G/2.04G [00:14<00:00, 153MB/s]


In [7]:
#Unzip the downloaded dataset
import zipfile
from zipfile import ZipFile
with ZipFile('plantvillage-dataset.zip', 'r') as z:
   z.extractall()

In [ ]:
base_dir = 'plantvillage dataset/color'

In [9]:
!pip install patchify

In [11]:
!pip install vit

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.2/297.2 kB 18.7 MB/s eta 0:00:00
  Created wheel for tasklib: filename=tasklib-2.5.1-py3-none-any.whl size=25472 sha256=2fd66ae359a66721d8f04265be87d62b3c28f5eb18f0241ab7fbae13670e3c19
  Stored in directory: /root/.cache/pip/wheels/06/97/8f/5067ac7e7a734bd4ffca090afa19a4498409f189cceb02cc1b
Successfully built tasklib


In [13]:
!pip install tensorflow_addons

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.4.1
    Uninstalling typeguard-4.4.1:
      Successfully uninstalled typeguard-4.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.5.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [41]:
%%writefile vit.py
import os

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

class ClassToken(Layer):
    def __init__(self):
        super().__init__()

    def build(self, input_shape):
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value = w_init(shape=(1, 1, input_shape[-1]), dtype=tf.float32),
            trainable = True
        )

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        hidden_dim = self.w.shape[-1]

        cls = tf.broadcast_to(self.w, [batch_size, 1, hidden_dim]) # change the shape
        cls = tf.cast(cls, dtype=inputs.dtype) # change the datatype
        return cls

def mlp(x, cf):
    x = Dense(cf["mlp_dim"], activation="gelu")(x)
    x = Dropout(cf["dropout_rate"])(x)
    x = Dense(cf["hidden_dim"])(x)
    x = Dropout(cf["dropout_rate"])(x)
    return x

def transformer_encoder(x, cf):
    skip_1 = x
    x = LayerNormalization()(x)
    x = MultiHeadAttention(num_heads=cf["num_heads"], key_dim=cf["hidden_dim"])(x, x)
    x = Add()([x, skip_1])

    skip_2 = x
    x = LayerNormalization()(x)
    x = mlp(x, cf)
    x = Add()([x, skip_2])

    return x

def ViT(cf):

    """ Inputs """
    input_shape = (cf["num_patches"], cf["patch_size"]*cf["patch_size"]*cf["num_channels"])
    print("input_shape",input_shape)
    inputs = Input(input_shape)     ## (None, 64, 1875)

    """ Patch + Position Embeddings """
    patch_embed = Dense(cf["hidden_dim"])(inputs)   ## (None, 64, 768)
    print("patch_embed",patch_embed.shape)
    positions = tf.range(start=0, limit=cf["num_patches"], delta=1)
    #print("positions",positions)
    pos_embed = Embedding(input_dim=cf["num_patches"], output_dim=cf["hidden_dim"])(positions) ## (64, 768)
    print("pos_embed",pos_embed.shape)
    embed = patch_embed + pos_embed ## (None, 64, 768)
    print("embed",embed.shape)


    """ Adding Class Token """
    token = ClassToken()(embed)
    print("token",token.shape)
    x = Concatenate(axis=1)([token, embed]) ## (None, 64, 768) # 65+1 token patch and 768 reduced dim
    print("x",x.shape)
    for _ in range(cf["num_layers"]):
        x = transformer_encoder(x, cf)

    """ Classification Head """
    x = LayerNormalization()(x)     ## (None, 65, 768)
    x = x[:, 0, :]
    x = Dense(cf["num_classes"], activation="softmax")(x)

    model = Model(inputs, x)
    return model


if __name__ == "__main__":
    config = {}
    config["num_layers"] = 6 #reduce num_layers from 12
    config["hidden_dim"] = 384 #reduce hidden_dim from 768
    config["mlp_dim"] = 1536 #reduce mlp_dim from 3072
    config["num_heads"] = 6 #reduce num_heads from 12
    config["dropout_rate"] = 0.1
    config["num_patches"] = 64
    config["patch_size"] = 25
    config["num_channels"] = 3
    config["num_classes"] = 38

    model = ViT(config)
    model.summary()

Overwriting vit.py


In [15]:
!pip install vit_keras

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.4 MB/s eta 0:00:00


In [36]:

import os
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from patchify import patchify
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping
from vit import ViT

""" Hyperparameters """
hp = {}
hp["image_size"] = 200
hp["num_channels"] = 3
hp["patch_size"] = 25
hp["num_patches"] = (hp["image_size"]**2) // (hp["patch_size"]**2)
hp["flat_patches_shape"] = (hp["num_patches"], hp["patch_size"]*hp["patch_size"]*hp["num_channels"])

hp["batch_size"] = 8
hp["lr"] = 1e-4
hp["num_epochs"] = 500
hp["num_classes"] = 38
hp["class_names"] = ['Apple___Apple_scab', 'Apple___Black_rot', 'Apple___Cedar_apple_rust',  'Apple___healthy',  'Blueberry___healthy', 'Cherry_(including_sour)___Powdery_mildew',  'Cherry_(including_sour)___healthy', 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',  'Corn_(maize)___Common_rust_',  'Corn_(maize)___Northern_Leaf_Blight',  'Corn_(maize)___healthy',  'Grape___Black_rot',  'Grape___Esca_(Black_Measles)',  'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',  'Grape___healthy',  'Orange___Haunglongbing_(Citrus_greening)',  'Peach___Bacterial_spot',  'Peach___healthy', 'Pepper,_bell___Bacterial_spot',  'Pepper,_bell___healthy',  'Potato___Early_blight',  'Potato___Late_blight',  'Potato___healthy',  'Raspberry___healthy',  'Soybean___healthy',  'Squash___Powdery_mildew',  'Strawberry___Leaf_scorch',  'Strawberry___healthy',  'Tomato___Bacterial_spot',  'Tomato___Early_blight',  'Tomato___Late_blight',  'Tomato___Leaf_Mold',  'Tomato___Septoria_leaf_spot',  'Tomato___Spider_mites Two-spotted_spider_mite',  'Tomato___Target_Spot',  'Tomato___Tomato_Yellow_Leaf_Curl_Virus',  'Tomato___Tomato_mosaic_virus',  'Tomato___healthy']


hp["num_layers"] = 12
hp["hidden_dim"] = 768
hp["mlp_dim"] = 3072
hp["num_heads"] = 12
hp["dropout_rate"] = 0.1

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_data(path, split=0.1):
    images = shuffle(glob(os.path.join(path, "*", "*.jpg")))

    split_size = int(len(images) * split)
    train_x, valid_x = train_test_split(images, test_size=split_size, random_state=42)
    train_x, test_x = train_test_split(train_x, test_size=split_size, random_state=42)

    return train_x, valid_x, test_x

def tf_dataset(images, batch=32):
    ds = tf.data.Dataset.from_tensor_slices((images))
    ds = ds.map(parse).batch(batch).prefetch(8)
    return ds

def process_image_label(path):
    """ Reading images """
    path = path.decode() # Convert tensor to string
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (hp["image_size"], hp["image_size"]))
    image = image/255.0

    """ Preprocessing to patches """
    patch_shape = (hp["patch_size"], hp["patch_size"], hp["num_channels"])
    patches = patchify(image, patch_shape, hp["patch_size"])
    print("patches",patches.shape)

    patches = np.reshape(patches, (64, 25, 25, 3))
    for i in range(64):
         cv2.imwrite(f"files/{i}.png", patches[i])

    patches = np.reshape(patches, hp["flat_patches_shape"])
    patches = patches.astype(np.float32)

    """ Label """
    class_name = path.split("\\")[-2]
    #print('class_name',class_name)
    class_idx = hp["class_names"].index(class_name)
    class_idx = np.array(class_idx, dtype=np.int32)

    return patches, class_idx

def parse(path):
    patches, labels = tf.numpy_function(process_image_label, [path], [tf.float32, tf.int32])
    labels = tf.one_hot(labels, hp["num_classes"])

    patches.set_shape(hp["flat_patches_shape"])
    labels.set_shape(hp["num_classes"])

    return patches, labels

if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory for storing files """
    create_dir("files")

    """ Paths """
    dataset_path = r'plantvillage dataset/color'

    model_path = os.path.join("files", "model.keras")
    csv_path = os.path.join("files", "log.csv")

    """ Dataset """
    train_x, valid_x, test_x = load_data(dataset_path)
    print(f"Train: {len(train_x)} - Valid: {len(valid_x)} - Test: {len(test_x)}")

    train_ds = tf_dataset(train_x, batch=hp["batch_size"])
    valid_ds = tf_dataset(valid_x, batch=hp["batch_size"])

    model = ViT(hp)

    model.compile(
        loss="categorical_crossentropy",
        optimizer=tf.keras.optimizers.Adam(hp["lr"], clipvalue=1.0),
        metrics=["acc"]
    )

    callbacks = [
        ModelCheckpoint(model_path, monitor='val_loss', verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, min_lr=1e-10, verbose=1),
        CSVLogger(csv_path),
        EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=False),
    ]
    """
    model.fit(
        train_ds,
        epochs=hp["num_epochs"],
        validation_data=valid_ds,
        callbacks=callbacks
    )
"""


Train: 1200 - Valid: 150 - Test: 150
input_shape (64, 1875)
patch_embed (None, 64, 768)
pos_embed (64, 768)
embed (None, 64, 768)
token (None, 1, 768)
x (None, 65, 768)


In [42]:
%%writefile train.py

import os
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from patchify import patchify
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping
from vit import ViT

""" Hyperparameters """
hp = {}
hp["image_size"] = 100 # Reduce image_size from 200
hp["num_channels"] = 3
hp["patch_size"] = 10 # Reduce patch_size from 25
hp["num_patches"] = (hp["image_size"]**2) // (hp["patch_size"]**2)
hp["flat_patches_shape"] = (hp["num_patches"], hp["patch_size"]*hp["patch_size"]*hp["num_channels"])

hp["batch_size"] = 4 # Reduce batch_size from 8
hp["lr"] = 1e-4
hp["num_epochs"] = 500
hp["num_classes"] = 38
hp["class_names"] = ['Apple___Apple_scab',  'Apple___Black_rot',  'Apple___Cedar_apple_rust',  'Apple___healthy', 'Blueberry___healthy',  'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy',  'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',  'Corn_(maize)___Common_rust_',  'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy',  'Grape___Black_rot',  'Grape___Esca_(Black_Measles)',  'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',  'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)',  'Peach___Bacterial_spot',  'Peach___healthy',  'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy',  'Potato___Early_blight',  'Potato___Late_blight',  'Potato___healthy',  'Raspberry___healthy',  'Soybean___healthy',  'Squash___Powdery_mildew',  'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot',  'Tomato___Early_blight',  'Tomato___Late_blight',  'Tomato___Leaf_Mold',  'Tomato___Septoria_leaf_spot',  'Tomato___Spider_mites Two-spotted_spider_mite',  'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus', 'Tomato___healthy']



hp["num_layers"] = 6 #reduce num_layers from 12
hp["hidden_dim"] = 384 #reduce hidden_dim from 768
hp["mlp_dim"] = 1536 #reduce mlp_dim from 3072
hp["num_heads"] = 6 #reduce num_heads from 12
hp["dropout_rate"] = 0.1

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_data(path, split=0.1):
    images = shuffle(glob(os.path.join(path, "*", "*.jpg")))

    split_size = int(len(images) * split)
    train_x, valid_x = train_test_split(images, test_size=split_size, random_state=42)
    train_x, test_x = train_test_split(train_x, test_size=split_size, random_state=42)

    return train_x, valid_x, test_x

def tf_dataset(images, batch=32):
    ds = tf.data.Dataset.from_tensor_slices((images))
    ds = ds.map(parse).batch(batch).prefetch(8)
    return ds

def process_image_label(path):
    """ Reading images """
    path = path.decode() # Convert tensor to string
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (hp["image_size"], hp["image_size"]))
    image = image/255.0

    """ Preprocessing to patches """
    patch_shape = (hp["patch_size"], hp["patch_size"], hp["num_channels"])
    patches = patchify(image, patch_shape, hp["patch_size"])
    #print("patches",patches.shape)

    patches = np.reshape(patches, (hp["num_patches"], hp["patch_size"], hp["patch_size"], hp["num_channels"]))
    #for i in range(hp["num_patches"]):
         #cv2.imwrite(f"files/{i}.png", patches[i])

    patches = np.reshape(patches, hp["flat_patches_shape"])
    patches = patches.astype(np.float32)

    """ Label """
    class_name = path.split("/")[-2] #fixed path
    #print('class_name',class_name)
    class_idx = hp["class_names"].index(class_name)
    class_idx = np.array(class_idx, dtype=np.int32)

    return patches, class_idx

def parse(path):
    patches, labels = tf.numpy_function(process_image_label, [path], [tf.float32, tf.int32])
    labels = tf.one_hot(labels, hp["num_classes"])

    patches.set_shape(hp["flat_patches_shape"])
    labels.set_shape(hp["num_classes"])

    return patches, labels

if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Directory for storing files """
    create_dir("files")

    """ Paths """
    dataset_path = r'plantvillage dataset/color'

    model_path = os.path.join("files", "model.keras")
    csv_path = os.path.join("files", "log.csv")

    """ Dataset """
    train_x, valid_x, test_x = load_data(dataset_path)
    print(f"Train: {len(train_x)} - Valid: {len(valid_x)} - Test: {len(test_x)}")

    train_ds = tf_dataset(train_x, batch=hp["batch_size"])
    valid_ds = tf_dataset(valid_x, batch=hp["batch_size"])

    model = ViT(hp)

    model.compile(
        loss="categorical_crossentropy",
        optimizer=tf.keras.optimizers.Adam(hp["lr"], clipvalue=1.0),
        metrics=["acc"]
    )

    callbacks = [
        ModelCheckpoint(model_path, monitor='val_loss', verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, min_lr=1e-10, verbose=1),
        CSVLogger(csv_path),
        EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=False),
    ]

    model.fit(
        train_ds,
        epochs=hp["num_epochs"],
        validation_data=valid_ds,
        callbacks=callbacks
    )

Overwriting train.py


In [44]:


import os
import numpy as np
import cv2
from patchify import patchify
import tensorflow as tf
from train import load_data, tf_dataset
from vit import ViT

""" Hyperparameters """
hp = {}
hp["image_size"] = 100 # Reduce image_size from 200
hp["num_channels"] = 3
hp["patch_size"] = 10 # Reduce patch_size from 25
hp["num_patches"] = (hp["image_size"]**2) // (hp["patch_size"]**2)
hp["flat_patches_shape"] = (hp["num_patches"], hp["patch_size"]*hp["patch_size"]*hp["num_channels"])

hp["batch_size"] = 4 # Reduce batch_size from 8
hp["lr"] = 1e-4
hp["num_epochs"] = 500
hp["num_classes"] = 38
hp["class_names"] = ['Apple___Apple_scab',  'Apple___Black_rot',  'Apple___Cedar_apple_rust',  'Apple___healthy', 'Blueberry___healthy',  'Cherry_(including_sour)___Powdery_mildew', 'Cherry_(including_sour)___healthy',  'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',  'Corn_(maize)___Common_rust_',  'Corn_(maize)___Northern_Leaf_Blight', 'Corn_(maize)___healthy',  'Grape___Black_rot',  'Grape___Esca_(Black_Measles)',  'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',  'Grape___healthy', 'Orange___Haunglongbing_(Citrus_greening)',  'Peach___Bacterial_spot',  'Peach___healthy',  'Pepper,_bell___Bacterial_spot', 'Pepper,_bell___healthy',  'Potato___Early_blight',  'Potato___Late_blight',  'Potato___healthy',  'Raspberry___healthy',  'Soybean___healthy',  'Squash___Powdery_mildew',  'Strawberry___Leaf_scorch', 'Strawberry___healthy', 'Tomato___Bacterial_spot',  'Tomato___Early_blight',  'Tomato___Late_blight',  'Tomato___Leaf_Mold',  'Tomato___Septoria_leaf_spot',  'Tomato___Spider_mites Two-spotted_spider_mite',  'Tomato___Target_Spot', 'Tomato___Tomato_Yellow_Leaf_Curl_Virus', 'Tomato___Tomato_mosaic_virus', 'Tomato___healthy']

hp["num_layers"] = 6 #reduce num_layers from 12
hp["hidden_dim"] = 384 #reduce hidden_dim from 768
hp["mlp_dim"] = 1536 #reduce mlp_dim from 3072
hp["num_heads"] = 6 #reduce num_heads from 12
hp["dropout_rate"] = 0.1


if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Paths """
    dataset_path = "/content/plantvillage dataset/color"
    model_path = os.path.join("/content/files", "model.keras") #fixed path to .keras

    """ Dataset """
    train_x, valid_x, test_x = load_data(dataset_path)
    print(f"Train: {len(train_x)} - Valid: {len(valid_x)} - Test: {len(test_x)}")

    test_ds = tf_dataset(test_x, batch=hp["batch_size"])

    """ Model """
    model = ViT(hp) # Use the same model object that was trained.
    model.load_weights(model_path)
    model.compile(
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
        optimizer=tf.keras.optimizers.Adam(hp["lr"]),
        metrics=["acc"]
    )

    model.evaluate(test_ds)

Train: 1200 - Valid: 150 - Test: 150
input_shape (100, 300)
patch_embed (None, 100, 384)
pos_embed (100, 384)
embed (None, 100, 384)
token (None, 1, 384)
x (None, 101, 384)


ResourceExhaustedError: {{function_node __wrapped__AddV2_device_/job:localhost/replica:0/task:0/device:GPU:0}} failed to allocate memory [Op:AddV2] name: 